In [6]:
import pulp

# Names of breweries
brewing = ["B1", "B2", "B3", "B4"]

# Max beer manufactured for each brewery
brewing_maximum = {"B1": 2000,
                   "B2": 2500,
                   "B3": 3500,
                   "B4": 2000}

# Dictionary for minimum units of beer from each brewery location
brewing_minimum = {"B1": 100,
                   "B2": 150,
                   "B3": 200,
                   "B4": 100}

# Create labels for packaging facilities
packaging = ["P1", "P2", "P3"]

# Packaging unit capacities
packaging_maximum = {"P1": 500,
                     "P2": 1500,
                     "P3": 2500}

# Aggregate demand for each DP to check solution allocations later on
total_dp_demand = {"DP1": 48, "DP2": 84, "DP3": 64, "DP4": 106, "DP5": 47,
               "DP6": 57, "DP7": 64, "DP8": 93, "DP9": 74, "DP10": 41,
               "DP11": 61, "DP12": 42, "DP13": 57, "DP14": 70, "DP15": 41}

# Min capacity at packaging units
packaging_minimum = {"P1": 50,
                     "P2": 100,
                     "P3": 150}

# Demand point labels
demand_points = ["DP1", "DP2", "DP3", "DP4", "DP5", "DP6", "DP7", "DP8", "DP9", "DP10", "DP11", "DP12", "DP13", "DP14", "DP15"]


total_demand = sum(total_dp_demand.values())

# Create a list of costs of transportation paths
# between breweries and packaging facilities

brew_to_pack_shipping_costs = [
    # packaging facilities in columns
    #  P1   P2   P3
    [1.55, 0.51, 0.90],  # Brewery 1
    [0.81, 3.18, 0.65],  # Brewery 2
    [2.13, 0.97, 0.51],  # Brewery 3
    [1.23, 2.15, 2.08]   # Brewery 4
]

# Create another list of transportation costs
# This one is between packaging facilities to demand points
pack_to_demand_shipping_costs = {
    "C1": {
        1: {1: 3.05, 2: 1.18, 3: 1.7},
        2: {1: 1.32, 2: 2.56, 3: 0.63},
        3: {1: 2.8, 2: 2.3, 3: 2.5},
        4: {1: 2.43, 2: 2.67, 3: 1.31},
        5: {1: 0.64, 2: 3.03, 3: 1.5},
        6: {1: 1.94, 2: 2.04, 3: 0.92},
        7: {1: 2.48, 2: 1.21, 3: 2.29},
        8: {1: 2.56, 2: 1.88, 3: 2.02},
        9: {1: 3.23, 2: 0.74, 3: 0.87},
        10: {1: 0.6, 2: 1.71, 3: 2.86},
        11: {1: 2.78, 2: 2.62, 3: 0.5},
        12: {1: 0.57, 2: 0.8, 3: 0.62},
        13: {1: 1.8, 2: 3.23, 3: 1.27},
        14: {1: 2.5, 2: 2.2, 3: 3.02},
        15: {1: 0.68, 2: 1.24, 3: 1.31},
    },
    "C2": {
        1: {1: 1.35, 2: 1.03, 3: 1.12},
        2: {1: 1.05, 2: 1.26, 3: 0.93},
        3: {1: 1.31, 2: 1.22, 3: 1.25},
        4: {1: 1.24, 2: 1.28, 3: 1.05},
        5: {1: 0.93, 2: 1.34, 3: 1.08},
        6: {1: 1.16, 2: 1.17, 3: 0.98},
        7: {1: 1.25, 2: 1.03, 3: 1.22},
        8: {1: 1.26, 2: 1.15, 3: 1.17},
        9: {1: 1.38, 2: 0.95, 3: 0.97},
        10: {1: 0.93, 2: 1.12, 3: 1.31},
        11: {1: 1.3, 2: 1.27, 3: 0.91},
        12: {1: 0.92, 2: 0.96, 3: 0.93},
        13: {1: 1.13, 2: 1.38, 3: 1.04},
        14: {1: 1.25, 2: 1.2, 3: 1.34},
        15: {1: 0.94, 2: 1.04, 3: 1.05},
    },
    "C3": {
        1: {1: 7.1, 2: 2.4, 3: 3.71},
        2: {1: 2.75, 2: 5.86, 3: 1.02},
        3: {1: 6.48, 2: 5.21, 3: 5.72},
        4: {1: 5.55, 2: 6.14, 3: 2.74},
        5: {1: 1.05, 2: 7.04, 3: 3.22},
        6: {1: 4.3, 2: 4.55, 3: 1.75},
        7: {1: 5.68, 2: 2.48, 3: 5.19},
        8: {1: 5.86, 2: 4.17, 3: 4.51},
        9: {1: 7.57, 2: 1.3, 3: 1.62},
        10: {1: 0.93, 2: 3.72, 3: 6.62},
        11: {1: 6.43, 2: 6.03, 3: 0.68},
        12: {1: 0.86, 2: 1.44, 3: 0.99},
        13: {1: 3.95, 2: 7.55, 3: 2.64},
        14: {1: 5.72, 2: 4.98, 3: 7.03},
        15: {1: 1.15, 2: 2.54, 3: 2.72},
    },
    "C4": {
        1: {1: 5.47, 2: 2, 3: 2.97},
        2: {1: 2.26, 2: 4.55, 3: 0.98},
        3: {1: 5.01, 2: 4.08, 3: 4.45},
        4: {1: 4.33, 2: 4.76, 3: 2.25},
        5: {1: 1, 2: 5.43, 3: 2.6},
        6: {1: 3.4, 2: 3.59, 3: 1.52},
        7: {1: 4.42, 2: 2.06, 3: 4.06},
        8: {1: 4.55, 2: 3.31, 3: 3.56},
        9: {1: 5.81, 2: 1.19, 3: 1.43},
        10: {1: 0.92, 2: 2.98, 3: 5.11},
        11: {1: 4.97, 2: 4.68, 3: 0.73},
        12: {1: 0.87, 2: 1.29, 3: 0.96},
        13: {1: 3.14, 2: 5.8, 3: 2.18},
        14: {1: 4.45, 2: 3.9, 3: 5.42},
        15: {1: 1.08, 2: 2.1, 3: 2.24},
    },
    "C5": {
        1: {1: 7.11, 2: 2.55, 3: 3.82},
        2: {1: 2.89, 2: 5.91, 3: 1.21},
        3: {1: 6.51, 2: 5.28, 3: 5.77},
        4: {1: 5.61, 2: 6.18, 3: 2.87},
        5: {1: 1.23, 2: 7.06, 3: 3.34},
        6: {1: 4.39, 2: 4.64, 3: 1.92},
        7: {1: 5.73, 2: 2.63, 3: 5.25},
        8: {1: 5.91, 2: 4.27, 3: 4.6},
        9: {1: 7.57, 2: 1.48, 3: 1.79},
        10: {1: 1.12, 2: 3.83, 3: 6.65},
        11: {1: 6.46, 2: 6.08, 3: 0.88},
        12: {1: 1.05, 2: 1.62, 3: 1.18},
        13: {1: 4.05, 2: 7.55, 3: 2.78},
        14: {1: 5.77, 2: 5.05, 3: 7.05},
        15: {1: 1.33, 2: 2.68, 3: 2.86},
    },
}


# Liquid Constraints
liquid_constraints = {
    1: {"Max Qty": 1000},
    2: {"Max Qty": 1000},
    3: {"Max Qty": 1000},
    4: {"Max Qty": 1000},
    5: {"Max Qty": 1500},
    6: {"Max Qty": 1500},
    7: {"Max Qty": 1500},
}


breweries_liquids_config = {
    "B1": {1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0},
    "B2": {1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 0},
    "B3": {1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 1},
    "B4": {1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 1},
}


commodities = {
    1: {"Liquid": 1, "Container": 1},
    2: {"Liquid": 1, "Container": 2},
    3: {"Liquid": 1, "Container": 4},
    4: {"Liquid": 2, "Container": 1},
    5: {"Liquid": 2, "Container": 3},
    6: {"Liquid": 2, "Container": 4},
    7: {"Liquid": 2, "Container": 5},
    8: {"Liquid": 3, "Container": 2},
    9: {"Liquid": 3, "Container": 5},
    10: {"Liquid": 4, "Container": 1},
    11: {"Liquid": 4, "Container": 4},
    12: {"Liquid": 4, "Container": 5},
    13: {"Liquid": 5, "Container": 1},
    14: {"Liquid": 5, "Container": 3},
    15: {"Liquid": 5, "Container": 4},
    16: {"Liquid": 5, "Container": 5},
    17: {"Liquid": 6, "Container": 2},
    18: {"Liquid": 6, "Container": 4},
    19: {"Liquid": 7, "Container": 1},
    20: {"Liquid": 7, "Container": 5},
}

demand = {
    1: {"DP1": 0, "DP2": 11, "DP3": 0, "DP4": 5, "DP5": 0, "DP6": 0, "DP7": 14, "DP8": 13, "DP9": 0, "DP10": 0, "DP11": 6, "DP12": 5, "DP13": 0, "DP14": 5, "DP15": 0},
    2: {"DP1": 0, "DP2": 0, "DP3": 0, "DP4": 12, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 0, "DP9": 12, "DP10": 0, "DP11": 13, "DP12": 0, "DP13": 0, "DP14": 12, "DP15": 7},
    3: {"DP1": 0, "DP2": 13, "DP3": 0, "DP4": 7, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 10, "DP9": 11, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 14, "DP15": 0},
    4: {"DP1": 0, "DP2": 0, "DP3": 0, "DP4": 9, "DP5": 0, "DP6": 0, "DP7": 7, "DP8": 7, "DP9": 0, "DP10": 10, "DP11": 0, "DP12": 10, "DP13": 8, "DP14": 0, "DP15": 0},
    5: {"DP1": 0, "DP2": 7, "DP3": 0, "DP4": 9, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 0, "DP9": 8, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 0, "DP15": 0},
    6: {"DP1": 0, "DP2": 0, "DP3": 0, "DP4": 13, "DP5": 0, "DP6": 10, "DP7": 0, "DP8": 9, "DP9": 0, "DP10": 0, "DP11": 14, "DP12": 0, "DP13": 12, "DP14": 0, "DP15": 0},
    7: {"DP1": 10, "DP2": 12, "DP3": 11, "DP4": 0, "DP5": 15, "DP6": 0, "DP7": 0, "DP8": 0, "DP9": 0, "DP10": 12, "DP11": 10, "DP12": 0, "DP13": 8, "DP14": 0, "DP15": 0},
    8: {"DP1": 9, "DP2": 0, "DP3": 0, "DP4": 0, "DP5": 6, "DP6": 0, "DP7": 5, "DP8": 0, "DP9": 0, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 8, "DP15": 0},
    9: {"DP1": 9, "DP2": 8, "DP3": 15, "DP4": 9, "DP5": 0, "DP6": 12, "DP7": 0, "DP8": 15, "DP9": 10, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 10, "DP14": 11, "DP15": 5},
    10: {"DP1": 7, "DP2": 0, "DP3": 0, "DP4": 0, "DP5": 0, "DP6": 0, "DP7": 14, "DP8": 0, "DP9": 0, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 0, "DP15": 0},
    11: {"DP1": 0, "DP2": 0, "DP3": 8, "DP4": 0, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 0, "DP9": 6, "DP10": 0, "DP11": 11, "DP12": 0, "DP13": 0, "DP14": 0, "DP15": 7},
    12: {"DP1": 0, "DP2": 10, "DP3": 0, "DP4": 0, "DP5": 0, "DP6": 0, "DP7": 11, "DP8": 0, "DP9": 0, "DP10": 6, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 0, "DP15": 0},
    13: {"DP1": 0, "DP2": 11, "DP3": 0, "DP4": 9, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 0, "DP9": 14, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 0, "DP15": 13},
    14: {"DP1": 0, "DP2": 12, "DP3": 0, "DP4": 8, "DP5": 8, "DP6": 11, "DP7": 0, "DP8": 0, "DP9": 0, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 8, "DP14": 0, "DP15": 0},
    15: {"DP1": 13, "DP2": 0, "DP3": 0, "DP4": 5, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 11, "DP9": 0, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 13, "DP15": 0},
    16: {"DP1": 0, "DP2": 0, "DP3": 0, "DP4": 9, "DP5": 0, "DP6": 0, "DP7": 7, "DP8": 0, "DP9": 0, "DP10": 0, "DP11": 0, "DP12": 14, "DP13": 0, "DP14": 0, "DP15": 0},
    17: {"DP1": 0, "DP2": 0, "DP3": 5, "DP4": 0, "DP5": 13, "DP6": 9, "DP7": 0, "DP8": 13, "DP9": 13, "DP10": 0, "DP11": 0, "DP12": 13, "DP13": 0, "DP14": 0, "DP15": 0},
    18: {"DP1": 0, "DP2": 0, "DP3": 10, "DP4": 11, "DP5": 5, "DP6": 6, "DP7": 6, "DP8": 0, "DP9": 0, "DP10": 13, "DP11": 0, "DP12": 0, "DP13": 0, "DP14": 7, "DP15": 0},
    19: {"DP1": 0, "DP2": 0, "DP3": 0, "DP4": 0, "DP5": 0, "DP6": 0, "DP7": 0, "DP8": 10, "DP9": 0, "DP10": 0, "DP11": 7, "DP12": 0, "DP13": 0, "DP14": 0, "DP15": 0},
    20: {"DP1": 0, "DP2": 0, "DP3": 15, "DP4": 0, "DP5": 0, "DP6": 9, "DP7": 0, "DP8": 5, "DP9": 0, "DP10": 0, "DP11": 0, "DP12": 0, "DP13": 11, "DP14": 0, "DP15": 0}
}


# Initialize model
prob = pulp.LpProblem("Beer_Distribution_Problem", pulp.LpMinimize)

# Make tuples for combinations of route 1 combos (brewery to packaging unit)
routes_1 = [(i, j) for i in brewing for j in packaging]

# Create dictionary for initial routes
inputs_1 = pulp.LpVariable.dicts("route", (brewing, packaging), 0, None, pulp.LpContinuous)

# Make tuples for combinations of route 2 combos (packaging units to DPs)
routes_2 = [(j, k) for j in packaging for k in demand_points]

# Create dictionary for second routes
inputs_2 = pulp.LpVariable.dicts("route", (packaging, demand_points), 0, None, pulp.LpContinuous)


# The objective function for all transportation costs from route 1 and route 2
prob += pulp.lpSum(
    [inputs_1[i][j] * brew_to_pack_shipping_costs[brewing.index(i)][packaging.index(j)] for i in brewing for j in packaging]
) + pulp.lpSum(
    [inputs_2[j][k] * pack_to_demand_shipping_costs[container][demand_points.index(k) + 1][packaging.index(j) + 1] for j in packaging for k in demand_points for container in pack_to_demand_shipping_costs]
), "All_Transportation_Costs"



# Demand constraints for each commodity at each demand point
for c in commodities:
    for k in total_dp_demand.keys():  
        required_qty = demand[c][k] if k in demand[c] else 0
        prob += pulp.lpSum([inputs_2[j][k] for j in packaging]) >= required_qty, f"Demand_Constraint_Commodity{c}_DemandPoint{k}"
        
        
        
# Brewery constraints for max capacity
for i in brewing:
    prob += pulp.lpSum([inputs_1[i][j] for j in packaging]) <= brewing_maximum[i], "Maximum_Brewing%s" % i

# Brewery constraints for min capacity
for i in brewing:
    prob += pulp.lpSum([inputs_1[i][j] for j in packaging]) >= brewing_minimum[i], "Minimum_Brewing%s" % i

# Constraints for max production at packaging facilities
for j in packaging:
    prob += pulp.lpSum([inputs_2[j][k] for k in demand_points]) <= packaging_maximum[j], "Maximum_Packaging%s" % j

# Constraints for min production at packaging facilities
for j in packaging:
    prob += pulp.lpSum([inputs_2[j][k] for k in demand_points]) >= packaging_minimum[j], "Minimum_Packaging%s" % j


# Constraint to ensure packaging unit output equals brewery output
for j in packaging:
    prob += pulp.lpSum([inputs_2[j][k] for k in demand_points]) == pulp.lpSum([inputs_1[i][j] for i in brewing]), "Packaging_Output%s" % j
    
    
# Brewery liquid commodity and container commodity constraints
for i in brewing:
    for c in commodities:
        required_liquid = commodities[c]["Liquid"]
        max_qty = liquid_constraints[required_liquid]["Max Qty"]
        # Add constraint only if the brewery uses the liquid (value = 1)
        if breweries_liquids_config[i][required_liquid] == 1:
            prob += pulp.lpSum([inputs_1[i][j] * commodities[c]["Container"] for j in packaging]) <= max_qty, f"Liquid_Constraint_Brewery{required_liquid}_{i}_Commodity{c}"
        else:
            # If the brewery doesn't use the liquid, the constraint will be 0 <= 0, which has no effect.
            prob += 0 <= 0, f"Liquid_Constraint_Brewery{required_liquid}_{i}_Commodity{c}"



# Solving the problem
prob.solve()

# Printing the results
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", round(v.varValue))

# Aggregating brewing and packaging totals
brewing_outputs = {i: sum(inputs_1[i][j].varValue for j in packaging) for i in brewing}
packaging_inputs = {j: sum(inputs_2[j][k].varValue for k in demand_points) for j in packaging}
packaging_outputs = {j: sum(inputs_2[j][k].varValue for k in demand_points) for j in packaging}

# Printing aggregated results
print("Brewing Outputs:")
for i, output in brewing_outputs.items():
    print(f"{i}: {output}")

print("Packaging Inputs:")
for j, input_ in packaging_inputs.items():
    print(f"{j}: {input_}")

print("Packaging Outputs:")
for j, output in packaging_outputs.items():
    print(f"{j}: {output}")

# Total demand and shipping costs
total_demand = sum(total_dp_demand.values())
total_shipping_costs = round(pulp.value(prob.objective))

print("Total Demand:", total_demand)
print("Total Shipping Costs:", total_shipping_costs)


# list of demand perturbations to analyze sensitivity
demand_perturbations = [0.5, 2, 5]

for perturbation in demand_perturbations:
    # Update demand values with perturbation
    perturbed_demand = {c: {k: int(v * perturbation) for k, v in demand[c].items()} for c in commodities}

    # Update demand constraints in the problem
    for c in commodities:
        for k in total_dp_demand.keys():
            required_qty = perturbed_demand[c][k] if k in perturbed_demand[c] else 0
            prob.constraints[f"Demand_Constraint_Commodity{c}_DemandPoint{k}"].rhs = required_qty

    # Solve the perturbed problem
    prob.solve()

    # Print the results
    print("\nResults for Demand Perturbation:", perturbation)
    print("Status:", pulp.LpStatus[prob.status])
    print("Objective Value:", round(pulp.value(prob.objective)))

    # Aggregating brewing and packaging totals for the perturbed problem
    brewing_outputs = {i: sum(inputs_1[i][j].varValue for j in packaging) for i in brewing}
    packaging_inputs = {j: sum(inputs_2[j][k].varValue for k in demand_points) for j in packaging}
    packaging_outputs = {j: sum(inputs_2[j][k].varValue for k in demand_points) for j in packaging}

    # Printing aggregated results
    print("Brewing Outputs:")
    for i, output in brewing_outputs.items():
        print(f"{i}: {output}")

    print("Packaging Inputs:")
    for j, input_ in packaging_inputs.items():
        print(f"{j}: {input_}")

    print("Packaging Outputs:")
    for j, output in packaging_outputs.items():
        print(f"{j}: {output}")

    # Total demand and shipping costs for the perturbed problem
    total_demand = sum(total_dp_demand.values())
    total_shipping_costs = round(pulp.value(prob.objective))

    print("Total Demand:", total_demand)
    print("Total Shipping Costs:", total_shipping_costs)
    

# Solving the problem
prob.solve()

# Printing the results
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", round(v.varValue))

# Aggregating brewing and packaging totals
brewing_outputs = {i: sum(inputs_1[i][j].varValue for j in packaging) for i in brewing}
packaging_inputs = {j: sum(inputs_2[j][k].varValue for k in demand_points) for j in packaging}
packaging_outputs = {j: sum(inputs_2[j][k].varValue for k in demand_points) for j in packaging}

# Printing aggregated results
print("Brewing Outputs:")
for i, output in brewing_outputs.items():
    print(f"{i}: {output}")

print("Packaging Inputs:")
for j, input_ in packaging_inputs.items():
    print(f"{j}: {input_}")

print("Packaging Outputs:")
for j, output in packaging_outputs.items():
    print(f"{j}: {output}")

# Total demand and shipping costs
total_demand = sum(total_dp_demand.values())
total_shipping_costs = round(pulp.value(prob.objective))

print("Total Demand:", total_demand)
print("Total Shipping Costs:", total_shipping_costs)

# Sensitivity Reports
for constraint in prob.constraints.values():
    shadow_price = constraint.pi
    slack = constraint.slack
    print(f"Constraint: {constraint.name}, Shadow Price: {shadow_price}, Slack: {slack}")
    
for variable in prob.variables():
    reduced_cost = variable.dj
    print(f"Variable: {variable.name}, Reduced Cost: {reduced_cost}")





Status: Optimal
route_B1_P1 = 0
route_B1_P2 = 100
route_B1_P3 = 0
route_B2_P1 = 0
route_B2_P2 = 0
route_B2_P3 = 150
route_B3_P1 = 0
route_B3_P2 = 0
route_B3_P3 = 200
route_B4_P1 = 100
route_B4_P2 = 0
route_B4_P3 = 0
route_P1_DP1 = 0
route_P1_DP10 = 13
route_P1_DP11 = 0
route_P1_DP12 = 59
route_P1_DP13 = 0
route_P1_DP14 = 0
route_P1_DP15 = 13
route_P1_DP2 = 0
route_P1_DP3 = 0
route_P1_DP4 = 0
route_P1_DP5 = 15
route_P1_DP6 = 0
route_P1_DP7 = 0
route_P1_DP8 = 0
route_P1_DP9 = 0
route_P2_DP1 = 13
route_P2_DP10 = 0
route_P2_DP11 = 0
route_P2_DP12 = 0
route_P2_DP13 = 0
route_P2_DP14 = 14
route_P2_DP15 = 0
route_P2_DP2 = 0
route_P2_DP3 = 15
route_P2_DP4 = 0
route_P2_DP5 = 0
route_P2_DP6 = 0
route_P2_DP7 = 14
route_P2_DP8 = 15
route_P2_DP9 = 29
route_P3_DP1 = 0
route_P3_DP10 = 0
route_P3_DP11 = 300
route_P3_DP12 = 0
route_P3_DP13 = 12
route_P3_DP14 = 0
route_P3_DP15 = 0
route_P3_DP2 = 13
route_P3_DP3 = 0
route_P3_DP4 = 13
route_P3_DP5 = 0
route_P3_DP6 = 12
route_P3_DP7 = 0
route_P3_DP8 = 0
ro